# DLC/ANIPOSE PIPELINE

Automatically converts fly videos to estimated 3D coordinates using DeepLabCut and Anipose

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from dlc import analyze_new
from preprocess import fix_point, remove_cols, df2hdf, gen_anipose_files
import utils

Loading DLC 2.3.5...


### STEP 0: Configuration

- Add filepath to DLC network config files (e.g `dlc_networks.yml`)
- Add filepath to folder with experiment videos

In [ ]:
# Filepath to the geontype folder containing videos
videos = r""
videos = Path(videos)

### STEP 1: Running DeepLabCut

This will run on a directory with fly video files and generate DLC pose estimations, outputting to the same directory.

##### Processing done:
- DeepLabCut `analyze_videos`
- DeepLabCut `filterpredictions`

In [ ]:
p_networks = Path(r'./common_files/dlc_networks.yml')
analyze_new(videos, p_networks)

### STEP 2: Preprocessing for anipose

##### Processing done:
- Fix points
- Remove columns
- Rename (GenotypeFly#-CamName)
- Convert to .h5
- Generate anipose file structure

In [ ]:
# TODO: will probably handle all 3 below from a single directory
error = False

# find all the CSVs that DLC generated
parent_dir = ""
for p_csv in utils.get_csvs(videos):
    parent_dir = p_csv.parent.parent.parent

    # Pick current generated, filtered CSV 
    
    # TODO: also check for cam name and model name
    csv_name = p_csv.stem
    if "filtered" not in csv_name:
        continue

    print(f"[INFO] Processing {p_csv.name}")
    csv_df = utils.load_csv_as_df(p_csv)

    # Fix points
    print("[INFO] Running `Fix points` preprocessing...")
    col_name = 'F-TaG' # Enter column name to match for removal 
    n = 1 # Values will be replaced with the nth entry. To replace with the mean, use n=0
    csv_df = fix_point(csv_df, col_name, n)

    # Remove cols 
    print("[INFO] Running `Remove cols` preprocessing...")
    start1 = 'L' # Remove col if start of name matches string
    start2 = 'R' # Remove col if start of name matches string
    end = '' # Remove col if end of name matches string
    csv_df = remove_cols(csv_df, start1, end)
    csv_df = remove_cols(csv_df, start2, end)

    # Rename and convert to .h5 (saves to original directory)
    print("[INFO] Converting CSV to HDF...")
    root = Path(r'\\mpfi.org\public\sb-lab\BallSystem_RawData')
    print(f"[INFO] Using root path {root}")
    if df2hdf(csv_df, p_csv, root): # error occured
        error = True


# Generate anipose file structure
p_calibration_target = Path(r"./common_files/calibration_target.yml") # calibration target config file 
p_calibration_timeline = Path(r"./common_files/calib_timeline.yml") # calibration timeline config file
calibration_type = utils.get_calibration_type(p_calibration_target, parent_dir)
if calibration_type == 'fly':
    p_anipose_config = Path(r"./common_files/config_fly.toml") # anipose config file
elif calibration_type == 'board':
    p_anipose_config = Path(r"./common_files/config_board.toml") # anipose config file
else:
    print(f"[ERROR] Invalid calibration type or calibration type not specified in {p_calibration_target}")
    error = True

# check that all the files exist
if not p_calibration_target.exists():
    print("[ERROR] `calibration_target.yml` does not exist.")
    error = True
if not p_calibration_timeline.exists():
    print("[ERROR] `calib_timeline.yml` does not exist.")
    error = True

if not error:
    print("[INFO] Generating anipose files...")
    gen_anipose_files(parent_dir, p_networks, p_anipose_config, p_calibration_target, p_calibration_timeline)

    print('Finished preprocessing...')
else:
    print("Terminated due to error")
